# Introduction of further File Formats
You'll learn all about the transformation from DICOM to different file formats.

This demo is a jupyter notebook, i.e. intended to be run step by step.

Author: Eric Einspänner
<br>
Contributor: Nastaran Takmilhomayouni

First version: 6th of July 2023


Copyright 2023 Clinic of Neuroradiology, Magdeburg, Germany

License: Apache-2.0

## Table of contents
0. [Initial Set-Up for Google Colab](#initial-set-up-for-google-colab)
1. [Initial Set-Up (offline)](#initial-set-up-offline)
2. [META](#meta)
3. [HDR](#hdr)
4. [MNC](#mnc)

## Initial Set-Up for Google Colab
<u> Execute these code blocks just in Google Colab! </u>

In [ ]:
!wget -q -O - https://github.com/University-Clinic-of-Neuroradiology/python-bootcamp/archive/refs/heads/main.tar.gz | tar -xzf - --strip-components=2 python-bootcamp-main/notebooks/DICOM

In [ ]:
import os
import sys
from google.colab import output
output.enable_custom_widget_manager()

sys.path.insert(0,'DICOM')
os.chdir(sys.path[0])

In [ ]:
%pip install -q ipympl numpy matplotlib SimpleITK nibabel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import imageio.v2 as imageio
import SimpleITK as sitk
import nibabel as nib

## Initial Set-Up (offline)

In [ ]:
# Make sure figures appears inline and animations works
# Edit this to "%matplotlib notebook" when using the "classic" jupyter notebook interface
%matplotlib widget

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import SimpleITK as sitk
import nibabel as nib

## --- Start notebook ---

The following function `format_and_render_plot()` is just a simplify formatting method for the plots in this notebook.

In [ ]:
def format_and_render_plot(axis=False, legend=False):
    '''
    Custom function to simplify common formatting operations for exercises. Operations include: 
    1. Turning off axis grids and legends, if not explicitly requested.
    2. Calling `plt.tight_layout` to improve subplot spacing.
    3. Calling `plt.show()` to render plot.
    '''
    fig = plt.gcf()
    for ax in fig.axes:
        if not axis:
            ax.axis('off')
        if legend:  
            ax.legend(loc='center right')  
    plt.tight_layout()
    plt.show()

## META
To read a META image/header file, you can use the SimpleITK library.

In [ ]:
mr_image = sitk.ReadImage('Data/training_001_mr_T1.mha')
npa = sitk.GetArrayViewFromImage(mr_image)

# Display the image slice from the middle of the stack, z axis
z = int(mr_image.GetDepth()/2)
npa_zslice = sitk.GetArrayViewFromImage(mr_image)[z,:,:]

fig, axes = plt.subplots(1, 3)
axes[0].imshow(npa_zslice)
axes[0].set_title('default colormap', fontsize=8, fontweight ="bold")
axes[1].imshow(npa_zslice, cmap='gray')
axes[1].set_title('grey colormap', fontsize=8, fontweight ="bold")
axes[2].imshow(npa_zslice, cmap='gray', vmin=npa.min(), vmax=npa.max())
axes[2].set_title('grey colormap,\n scaling based on\n volumetric min and max values', fontsize=8, fontweight ="bold")
format_and_render_plot()

## HDR
To work with HDR image, you can use python imageio and/or nibabel package.

HDR files can come with both header (.hdr) and image files (.img) or only the image file.

In order to load HDR image file through header, you can use `nibabel`:

In [ ]:
img = nib.load(img_path)
hdr = nib.load(hdr_path)

In [ ]:
print(img)

To convert hdr file you can use load function from python nibabel module to load the image and then save funtion to save it as nifti.

Note: both img and hdr files should be in the folder but you can work with .img file

In [ ]:
nib.save(img, img_path.replace('.img', '.nii'))

You can also convert .nii to .hdr and .img file will be automatically created!

In [ ]:
fname = 'Data/OAS1_0001_MR1_mpr_n4_anon_111_t88_masked_gfc_fseg.nii'
img = nib.load(fname)

print(type(img))
print(img.affine)

nib.nifti1.save(img, fname.replace('.nii','.hdr'))

## MNC

In [ ]:
img = nib.load('Data/subject04.mnc')

data = img.get_fdata()
print(data.mean())
print(data.max())
print(data.min())
print(type(data))
print(np.shape(data))

In [ ]:
affine = np.array([[0, 0, 1, 0],
                   [0, 1, 0, 0],
                   [1, 0, 0, 0],
                   [0, 0, 0, 1]])

out = nib.Nifti1Image(img.get_fdata(), affine=affine)
nib.save(out, 'Data/subject04.nii.gz')

In [ ]:
print(type(img))
print(type(out))

In [ ]:
minc_data = img.get_fdata()
out_data = out.get_fdata()

fig, axes = plt.subplots(1, 2)
axes[0].imshow(minc_data[:,100,:], cmap='gray')
axes[0].set_title('minc', fontweight ="bold")
axes[1].imshow(out_data[:,100,:], cmap='gray')
axes[1].set_title('Nifti', fontweight ="bold")
format_and_render_plot()